## Day 3
Url: https://adventofcode.com/2022/day/3

Problem: each line of the input (`'./inputs/day3.txt'`) contains the inventory of items in a elf's backpack. The first half of the line is in one compartment of the backpack, the second half is in a second compartment. The elf who packed the items did a mistake and packed one item in both compartments for every elf. 

We need to find:
1) which was the misplaced element for each elf, and
2) assign to each misplaced element a score (given by the problem)
3) calculate the total score of all the misplaced elements

Scoring:
a to z: 1 to 26
A to Z: 27 to 52


In [1]:
import pandas as pd
import numpy as np
import string

In [2]:
inventory = pd.read_csv('./inputs/day3.txt', header=None)
inventory.columns = ['list_of_items']
inventory['misplaced_item'] = None
inventory['assigned score'] = 0

We write a couple of functions to find and calculate the value of the item:

In [3]:
def assign_value_to_item(item):
    point_mapping = pd.DataFrame({'letters': list(string.ascii_lowercase + string.ascii_uppercase),
              'values' : range(1, 53)})
    matching_value = point_mapping.loc[point_mapping['letters'] == item]['values'].values

    return matching_value

def find_misplaced_item(list_of_items):
    number_of_items = len(list_of_items)
    break_index = int(number_of_items/2)
    in_first_compartment = list_of_items[:break_index]
    in_second_compartment = list_of_items[break_index:]
    assert(len(in_first_compartment) == len(in_second_compartment))

    # use set() to find commonalities:
    common_item = list( set(in_first_compartment) & set(in_second_compartment) )
    
    return common_item[0]


Now we assign this function column wise:

In [4]:
inventory['misplaced_item'] = inventory.apply(lambda row: find_misplaced_item(row['list_of_items']), axis=1)
inventory['assigned score'] = inventory.apply(lambda row: assign_value_to_item(row['misplaced_item']), axis=1)


Now we can finally sum on the `assigned score` column.

In [5]:
total_value = inventory['assigned score'].sum()
print(f'The total value of misplaced items is {total_value}')

The total value of misplaced items is [8072]


That is the correct answer!

## Part 2
These eleves are starting to piss me off, truly. Now the problem is finding the common element in backpacks 3 by 3 (that is, each three lines correspond to the same group of elves, which is showed by the only element they have in common). Find the value of the sum of these badges.

In [6]:
inventory['group_id'] = 0
inventory['group_badge'] = None
inventory['assigned_for_badge'] = 0

In [7]:
count_id = 0
for i in range(0, len(inventory)):
    if not (i % 3):
        count_id += 1
    inventory['group_id'][i] = count_id
    

/tmp/ipykernel_405484/3590184507.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inventory['group_id'][i] = count_id


In [8]:
def assign_badge_by_id(group_id):
    elves_in_group = inventory[inventory['group_id']==group_id]
    element_in_common = set(elves_in_group['list_of_items'].iloc[0]) & \
                        set(elves_in_group['list_of_items'].iloc[1]) & \
                        set(elves_in_group['list_of_items'].iloc[2])
    badge = list(element_in_common)[0]
    return badge

In [9]:
inventory['group_badge'] = inventory.apply(lambda x: assign_badge_by_id(x['group_id']), axis=1)
inventory['assigned_for_badge'] = inventory.apply(lambda x: assign_value_to_item(x['group_badge']), axis=1)

Finally, the sum of the column `assigned_for_badge` will give us the number we want, but since it sums over all elves, we need to divide the number by 3!

In [10]:
total_score_for_badges = inventory['assigned_for_badge'].sum()/3
print(f'The total score by group is {int(total_score_for_badges)}.')

The total score by group is 2567.


And this is also the correct answer. Quite happy about this, I'm sure there would be better ways to apply these functions to all the columns than looping through it, but for now this will work.